<center>
    <h1>Implementation and Analysis of a SAT Solver</h1>
    <h2>Aidan Plunkitt</h2>
    <br />
</center>



# Introduction

The Boolean satisfiability problem, often abbreviated as SAT, is the problem of determining if there is at least one solution to a given Boolean formula. A solution is some assignment of Boolean variables (either True or False) that results in the formula evaluating to True.

SAT is significant because it was the first problem proven to be NP-complete, meaning solutions may only be found via some brute force search algorithm (exponential time complexity) but valid solutions can be quickly verified (polynomial time complexity). Other problems can be shown to be NP-complete if they can be mathematically reduced to the SAT problem. Thus, algorithms to quickly and efficiently find solutions to a boolean formula (aptly named SAT solvers) are useful across a wide range of optimization problems.

Any Boolean formula may be converted into a logically equivalent formula that is in Conjunctive Normal Form (CNF). An example of a formula in CNF is `(A OR B) AND (C OR D OR E) AND (F)`. Each letter is a `literal`, each disjunction (OR) of literals is known as a `clause`, and the entire formula is a conjunction (AND) of such clauses. Nearly every existing SAT solver and automated theorem prover assumes the input to be in CNF form, as does this implementation.


## The algorithm

A naive brute-force algorithm can determine whether a given formula is satisfiable; simply test every combination of True/False for the entire set of variables in the formula. A pure brute-force can be improved upon by utilizing "backtracking", where part of a solution space is no longer explored once it is shown to be unsatisfiable.

The pseudocode of a backtracking search algorithm is as follows:

```
backtrack(formula):
    for each variable v_i in the formula:
        if v_i has not yet been assigned a value:
            v_i = True
            check_sat(formula)
            
            if not UNSAT:
                result = backtrack(formula)
            
            v_i = False
            check_sat(formula)
            
            if not UNSAT:
                result = backtrack(formula)
                
    return result 
```  



The Davis-Putnam-Logemann-Loveland algorithm (DPLL) is a backtracking-based search algorithm that adds two further enhancements to avoid large sections of the naive search space. 
* Unit propogation looks for clauses with a single literal. Since every clause must be SAT for the formula to be SAT, then by extension the literal of a unit clause must be SAT for the formula to be SAT. Thus, the variable associated with that literal can be assigned a value (True/False) and the unit clause can be discarded.
* Pure literal elimination looks for variables that only have a single polarity (either True or False) in every instance in the formula, which are known as pure. These pure literals can always be made true, so they may be discarded from the search space.

# Implementation

Below is a basic SAT solver implemented in Python, both with and without optimization. You can see that the optimized `dpll` function first filters unit clauses from the formula before calling the `backtrack` function.

In [1]:
# basic backtracking algorithm without DPLL heuristics applied
def backtrack(f):
    # Try valuing the next unvalued variable.
    for v in f.vars:
        # if value has not yet been assigned
        if v.value == None:
            for value in [True, False]:   # 2 possible values
                v.value = value
                if not f.unsat():
                    r = backtrack(f)     # recurrence: T(n) = 2T(n-1) + p, 
                                          # where p is some first-degree polynomial
                                          # p = O(c * l), where c is the number of clauses
                                          # and l is the number of literals given in the input
                    if r != None:
                        return r
                # else backtrack
            v.value = None
            return None

    # All variables have been valued. Check to see if it's a solution
    if f.sat():
        return f.vars
    return None

# Simplified Davis–Putnam–Logemann–Loveland (DPLL) algorithm
# includes unit propogation but no pure literal elimination
def dpll(f):
    # unit propogation
    unit_clauses = []
    for c in f.clauses:
        # look for unit clauses
        if len(c) == 1:
            lit = c.lits[0]
            curr_val = f.vars[int(lit.var.name) - 1].value
            if curr_val and (curr_val != lit.sign):
                # cannot be satisfied
                return None
            # variable must be this value in order for f to be sat
            # this will reduce the number of recursive calls needed below
            f.vars[int(lit.var.name) - 1].value = lit.sign
            unit_clauses.append(c)
            
    # remove unecessary unit clauses to improve speedup
    f.clauses = [x for x in f.clauses if x not in unit_clauses]
    
    # call backtracking algorithm with improvements
    return backtrack(f)

The following helper classes and functions are required by the main algorithm:

In [2]:
class Var(object):
    def __init__(self, name, value=None):
        self.name = name
        self.value = value

    def __str__(self):
        return str(self.name)

class Lit(object):
    def __init__(self, var, sign):
        self.var = var
        self.sign = sign

    # Returns True iff the literal evaluates positively (and is not None)
    def sat(self):
        return self.var.value == self.sign

    # Returns True iff the literal evaluates negatively (or is None)
    def unsat(self):
        if self.var.value == None or self.sat():
            return False
        return True

    def __str__(self):
        name = str(self.var)
        if self.sign:
            return name
        return "-" + name

class Clause(object):
    def __init__(self, lits):
        self.lits = lits

    # Returns True if *some* literal in the clause evaluates positively
    # (T v F v F ...) == T
    def sat(self):
        for l in self.lits:
            if l.sat():
                return True
        return False

    # Returns True if *every* literal in the clause evaluates
    # (F v F v F) == F
    def unsat(self):
        for l in self.lits:
            if not l.unsat():
                return False
        return True

    def __str__(self):
        return " ".join([str(l) for l in self.lits]) + " 0"
    
    def __len__(self):
        return len(self.lits)

class Formula(object):
    def __init__(self, vars, clauses):
        self.vars = vars
        self.clauses = clauses

    # Returns True iff *every* clause in the formula is sat
    def sat(self):
        for c in self.clauses:
            if not c.sat():
                return False
        return True

    # Returns True iff *some* clause in the formula is unsat
    def unsat(self):
        for c in self.clauses:
            if c.unsat():
                return True
        return False
        
    def __str__(self):
        comment = "c Formula in DIMACS format\n"
        header = "p cnf {} {}\n".format(len(self.vars), len(self.clauses))
        clauses = "\n".join([str(c) for c in self.clauses])
        return comment + header + clauses

    
# Read a CNF formula from the given file in DIMACS format
def read_formula(filename):
    with open(filename, "r") as f:
        # Strip leading comments.
        while True:
            header = next(f).strip()
            if header[0] != 'c':
                break

        # Parse header and create variables.
        hfields = header.split()
        assert len(hfields) == 4
        assert hfields[0] == 'p'
        assert hfields[1] == 'cnf'
        nvars = int(hfields[2])
        nclauses = int(hfields[3])
        vars = [Var(name + 1) for name in range(nvars)]

        clauses = []
        for row in f:
            cvars = row.split()
            ncvars = len(cvars)
            assert int(cvars[ncvars - 1]) == 0
            clause = []
            for i in range(ncvars - 1):
                cvar = int(cvars[i])
                assert cvar != 0
                sign = cvar > 0
                cvar = abs(cvar)
                clause.append(Lit(vars[cvar - 1], sign))
            clauses.append(Clause(clause))

        return Formula(vars, clauses)

    
# Print a solution in DIMACS format
def print_soln(soln):
    if soln == None:
        print("s UNSATISFIABLE")
    else:
        print("s SATISFIABLE")
        asgs = ["v"]
        for v in soln:
            if v.value == True:
                asgs.append(str(v))
            elif v.value == False:
                asgs.append("-" + str(v))
        asgs.append("0")
        print(" ".join(asgs))

Below, both the `backtrack` and the optimized `dpll` are called on a simple three-variable Boolean function to demonstrate usage. See the sat3.dimacs file for the input used, but it is summarized in the code comments below.

In [3]:
# Demonstrate simple test with 3 variables, encoded in DIMACS format in sat3.dimacs
#
#  (v1) AND (v2 OR v3) AND (!v3)
#  1
#  2  3
# -3

f = read_formula("sat3.dimacs")

r1 = backtrack(f)
print_soln(r1)

r2 = dpll(f)
print_soln(r2)

s SATISFIABLE
v 1 2 -3 0
s SATISFIABLE
v 1 2 -3 0


# Theoretical Analysis

SAT, like all NP problems, has a theoretically exponential time complexity. As commented in the code of the `backtrack` function, the implementation of this algorithm has a recurrence relation of `T(n) = 2T(n-1) + p`. This can be visualized as a binary tree of height n, thus there are 2<sup>n</sup> nodes and the algorithm is O(2<sup>n</sup>). However, like dynamic programming, SAT solvers aim to reduce the size of the solution space that actually needs to be checked. Additionally, in practice SAT solvers may employ heuristics that make certain assumptions about the input data. This can significantly speed up common cases to quickly solve otherwise intractable problems with exponentially asymptotic runtimes.

# Empirical Analysis

Below, a Boolean formula with 50 variables is provided as input for both the basic backtracking algorithm implementation as well as the more optimized algorithm.

In [4]:
from time import time

f = read_formula("sat50.dimacs")

ts = time()
r1 = backtrack(f)
t1 = time() - ts

ts = time()
r2 = dpll(f)
t2 = time() - ts

print_soln(r1)
print_soln(r2)

print()
print("Backtrack implementation time:\t",t1)
print("DPLL implementation time:\t",t2)

s SATISFIABLE
v 1 2 3 4 5 6 -7 8 9 -10 -11 -12 13 -14 15 16 -17 -18 -19 20 -21 -22 23 24 -25 26 27 -28 -29 30 31 32 -33 -34 -35 36 37 -38 39 40 41 42 43 -44 45 -46 -47 48 49 -50 0
s SATISFIABLE
v 1 2 3 4 5 6 -7 8 9 -10 -11 -12 13 -14 15 16 -17 -18 -19 20 -21 -22 23 24 -25 26 27 -28 -29 30 31 32 -33 -34 -35 36 37 -38 39 40 41 42 43 -44 45 -46 -47 48 49 -50 0

Backtrack implementation time:	 12.100706100463867
DPLL implementation time:	 0.00018310546875


# Conclusion

While NP problems are the bane of computer scientists, there are clever heuristics that can be used to significantly narrow the actual amount of computation that needs to be done. In this case, the simple additions of unit propogation and backtracking to a brute force search algorithm provide a substantial realized runtime speedup. The difference is several orders of magnitude despite the theoretical complexity of the algorithm not changing. Further improvements could be made as well, including an implementation of pure literal elimination as well as "hot-loop" optimization of the code that is executed between each recurrence.